# Detection:

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
mp_holistic=mp.solutions.holistic
mp_drawing=mp.solutions.drawing_utils

def mediapipe_detection(image,model):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable=False
    results=model.process(image)
    image.flags.writeable=True
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results

def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)

with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    cap=cv2.VideoCapture(0)
    while cap.isOpened():
        res, frame=cap.read()
        image, results = mediapipe_detection(frame, holistic)
        draw_landmarks(image,results)
        cv2.imshow("Frame",image)
        if cv2.waitKey(30)&0xff==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# File Setup

In [ ]:
import os
DATA_PATH=os.path.join("MP_Data")
signs=(['Hello','Thanks','Friends'])
no_sequences=30
sequence_length=30
for sign in signs:
    for sequence in range(no_sequences):
        os.makedirs(os.path.join(DATA_PATH, sign,str(sequence)))

# Data Collection And Training

In [ ]:
import cv2
import mediapipe as mp
from matplotlib import pyplot as plt
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, LSTM
#from tensorflow.keras.callbacks import TensorBoard
mp_holistic=mp.solutions.holistic
mp_drawing=mp.solutions.drawing_utils

def mediapipe_detection(image,model):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable=False
    results=model.process(image)
    image.flags.writeable=True
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results

def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)


def extract_keypoints(results):
    pose=[]
    face=[]
    lh=[]
    rh=[]
    try:
        for res in results.pose_landmarks.landmark:
            pose.append([res.x,res.y,res.z,res.visibility])
        pose=np.array(pose).flatten()
    except:
        pose=np.zeros(132)
    try:
        for res in results.face_landmarks.landmark:
            face.append([res.x,res.y,res.z])
        face=np.array(face).flatten()
    except:
        face=np.zeros(1404)
    
    try:
        for res in results.left_hand_landmarks.landmark:
            lh.append([res.x,res.y,res.z])
        lh=np.array(lh).flatten()
    except:
        lh=np.zeros(63)
    
    try:
        for res in results.right_hand_landmarks.landmark:
            rh.append([res.x,res.y,res.z])
        rh=np.array(rh).flatten()
    except:
        rh=np.zeros(63)

    return np.concatenate([pose,face,lh,rh])

DATA_PATH=os.path.join("MP_Data")
signs=(['Hello','Thanks','Friends'])
no_sequences=30
sequence_length=30
label_map={label:num for num, label in enumerate(signs)}
sequences, labels=[],[]
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    cap=cv2.VideoCapture(0)
    for sign in signs:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length): 
                if frame_num==0:
                    res, frame=cap.read()
                    image, results = mediapipe_detection(frame, holistic)
                    draw_landmarks(image,results)
                    cv2.putText(image, 'Collection starting',(120,200),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
                    cv2.putText(image, 'Collection FOR {} Video No. {}'.format(sign,sequence),(15,20),cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,255),4,cv2.LINE_AA)
                    cv2.imshow("Frame",image)
                    cv2.waitKey(2000)
                else:
                    res, frame=cap.read()
                    image, results = mediapipe_detection(frame, holistic)
                    draw_landmarks(image,results)
                    cv2.putText(image, 'Collection FOR {} Video No. {}'.format(sign,sequence),(15,20),cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,255),4,cv2.LINE_AA)
                    cv2.imshow("Frame",image)
                    cv2.waitKey(1)

                keypoints=extract_keypoints(results)
                npy_path=os.path.join(DATA_PATH, sign,str(sequence),str(frame_num))
                #np.save(npy_path,keypoints)
                sequences.append(keypoints)
                labels.append(label_map[sign])
    cap.release()
    cv2.destroyAllWindows()
x=np.array(sequences)

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import TensorBoard
log_dir = os.path.join('logs')
x=np.array(sequences)
y=to_categorical(labels).astype(int)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.05)
np.save(x)
np.save(y)
log_dir = os.path.join('logs')
tb_callback=TensorBoard(log_dir=log_dir)
model=Sequential()
model.add(LSTM(64,True,input_shapes(30,1662),activation='relu'))
model.add(LSTM(64,True,activation='relu'))
model.add(LSTM(64,False,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(signs.shape[0],activation='softmax'))


In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrices='categorical_accuracy')
model.fit(x_train,y_train,epochs=200,callbacks=[tb_callback])
model.save('signs.h5')

In [ ]:
import cv2
import mediapipe as mp
from matplotlib import pyplot as plt
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, LSTM
#from tensorflow.keras.callbacks import TensorBoard
mp_holistic=mp.solutions.holistic
mp_drawing=mp.solutions.drawing_utils

def mediapipe_detection(image,model):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable=False
    results=model.process(image)
    image.flags.writeable=True
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results

def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)


def extract_keypoints(results):
    pose=[]
    face=[]
    lh=[]
    rh=[]
    try:
        for res in results.pose_landmarks.landmark:
            pose.append([res.x,res.y,res.z,res.visibility])
        pose=np.array(pose).flatten()
    except:
        pose=np.zeros(132)
    try:
        for res in results.face_landmarks.landmark:
            face.append([res.x,res.y,res.z])
        face=np.array(face).flatten()
    except:
        face=np.zeros(1404)
    
    try:
        for res in results.left_hand_landmarks.landmark:
            lh.append([res.x,res.y,res.z])
        lh=np.array(lh).flatten()
    except:
        lh=np.zeros(63)
    
    try:
        for res in results.right_hand_landmarks.landmark:
            rh.append([res.x,res.y,res.z])
        rh=np.array(rh).flatten()
    except:
        rh=np.zeros(63)

    return np.concatenate([pose,face,lh,rh])
sequences=[]
sentences=[]
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    cap=cv2.VideoCapture(0)
    while cap.isOpened():
        res, frame=cap.read()
        image, results = mediapipe_detection(frame, holistic)
        draw_landmarks(image,results)
        keypoints=extract_keypoints(results)
        sequence.append(keypoints)
        sequence=sequence[:30]
         if len(sequence)==30:
            res=model.predict(np.expand_dims(sequence, axis=0))[0]
        sentences.append(signs[np.argmax(res)])
        sentences=sentences[-5:]
        cv2.putText(image, '{}'.format(sentences),(120,200),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
        cv2.imshow("Frame",image)
        if cv2.waitKey(30)&0xff==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()